In [1]:
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import datetime as dt
import time

In [2]:
# portfolio of stocks
# stocks = ['VICI', 'JOE', 'ELS', 'IIPR-PA', 'G24.DE'] # real_estate
stocks = ['RACE', 'SMCI', 'CRM', 'DIS', 'ACN', 'PINS', 'NVO', 'ADBE', 'LULU', 'PYPL', 'MU'] # carbon finance
# stocks = ['2897.T', 'CLX', 'NATR', 'GRUMAB.MX', 'CMG', 'ADS.DE', 'LULU', 'DECK'] # consumer_discretion 
# stocks = ['RACE', 'LVS', 'MRK', 'NIO', 'PYPL', 'ADBE', 'FTNT', 'CL', 'ZTS', 'LDOS', 'MNST', 'ADSK', 'MDLZ', 'ISRG', 'MRK', 'NVO', 'UNH', '4528.T', '4568.T', 'BRO', 'DEMANT.CO', 'HRMY', 'HSTM', 'TWLO', 'CRM', 'DUOL', 'GDDY', 'ACN', 'TPL', 'DOCU']

#### Download historical data from yf API

In [3]:
import yfinance as yf

start_date = "2020-01-01"
end_date = datetime.now().strftime("%Y-%m-%d")

# Download historical data from yf API
data = yf.download(stocks, start=start_date, end=end_date, group_by='ticker')

# download data to csv
# data.to_csv("stocks.csv")
# data.to_csv("stocks_real_estate.csv")
data.to_csv("stocks_consumer_discretion.csv")

[*********************100%***********************]  11 of 11 completed


#### Download analysis from csv downloaded data

In [4]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
# df = pd.read_csv("stocks.csv", header=[0,1], index_col=0)
df = pd.read_csv("stocks_consumer_discretion.csv", header=[0,1], index_col=0)
# df = pd.read_csv("stocks_real_estate.csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Forward fill missing values for each column
df = df.fillna(method='ffill')

# Convert all values to float
df = df.astype(float)

# Show the result
df.head()

Ticker           PINS                                               SMCI  \
Price            Open       High        Low      Close     Volume   Open   
Date                                                                       
2020-01-02  18.799999  19.100000  18.565001  18.799999  6334600.0  2.385   
2020-01-03  18.600000  18.709999  18.254999  18.360001  5203200.0  2.340   
2020-01-06  18.270000  19.350000  18.219999  18.910000  7978900.0  2.340   
2020-01-07  18.879999  19.420000  18.760000  19.260000  7471500.0  2.301   
2020-01-08  19.250000  19.980000  19.115000  19.719999  7363500.0  2.305   

Ticker                                      ...        NVO             \
Price        High    Low  Close     Volume  ...       Open       High   
Date                                        ...                         
2020-01-02  2.418  2.342  2.363  4178000.0  ...  26.487128  26.528010   
2020-01-03  2.374  2.340  2.344   302000.0  ...  25.851178  26.073760   
2020-01-06  2.350  2.323  2.327   933000.0  ...  25.887521  25.973827   
2020-01-07  2.345  2.301  2.310   387000.0  ...  25.892059  25.892059   
2020-01-08  2.320  2.300  2.307  3537000.0  ...  25.851179  25.978367   

Ticker                                             RACE              \
Price             Low      Close     Volume        Open        High   
Date                                                                  
2020-01-02  26.350853  26.509840  2224800.0  160.378665  162.033814   
2020-01-03  25.828466  25.942028  2491200.0  160.100212  161.050680   
2020-01-06  25.833010  25.896605  2495000.0  158.180127  159.649024   
2020-01-07  25.692191  25.869347  2080600.0  159.351393  159.485801   
2020-01-08  25.742159  25.823923  1826200.0  160.369063  162.221993   

Ticker                                        
Price              Low       Close    Volume  
Date                                          
2020-01-02  159.975440  161.780350  195800.0  
2020-01-03  159.627871  160.071411  134600.0  
2020-01-06  158.180127  159.226608  169900.0  
2020-01-07  158.583339  158.621750  309300.0  
2020-01-08  160.090650  161.809158  261400.0  

[5 rows x 55 columns]

In [5]:
# # Here you can plot the charts for every ticker

# # For every ticker, plot Close
# import matplotlib.pyplot as plt

# for ticker in df.columns.levels[0]:
#     plt.figure(figsize=(12, 6))
#     plt.plot(df[ticker]['Close'], label=ticker)
#     plt.title(f"{ticker} Close Price")
#     plt.xlabel("Date")
#     plt.ylabel("Close Price")
#     plt.legend()
#     plt.show()

#### Plot of Volume Profile for All Stocks

In [6]:
import plotly.io as pio
pio.renderers.default = 'vscode'
import plotly.express as px
import plotly.graph_objects as go

for stock in stocks:
    # Use the stock data from our df
    df_stock = df[stock][['Close', 'Volume']]

    # Convert the stock to datetime if needed
    if not isinstance(df_stock.index, pd.DatetimeIndex):
        df_stock.index = pd.to_datetime(df_stock.index)

    # plot
    hist = go.Histogram(
        y=df_stock['Close'],
        x=df_stock['Volume'],
        nbinsy=150,
        orientation='h',
        name='Volume Profile',
        marker_color='#B0C4DE',
        opacity=0.5,
        xaxis='x2',
    )

    line = go.Scatter(
        x=df_stock.index,
        y=df_stock['Close'],
        mode='lines',
        name='Close',
        marker_color='blue',
        xaxis='x',
        yaxis='y',
    )

    fig = go.Figure([hist, line])

    fig.update_layout(
        xaxis=dict(
            title='Date',
            anchor='y',
            domain=[0, 1],
            showgrid=False,
            side='bottom'
        ),
        xaxis2=dict(
            title='Sum of Volume',
            overlaying='x',
            side='top',
            showgrid=False
        ),
        yaxis=dict(
            title='Close Price',
            showgrid=True
        ),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        ),
        height=600,
        title=f'Volume Profile & {stock} Close Prices'
    )

    fig.show()